### Notes on the data

- There are 6598 rows (approximately 20% of total number of records) that contain a value for "ENERGY STAR Score"
- Relatively uniform distribution of records from 2013 - 2019. Smaller # of records for 2011 and 2012
- The columns "Percent Better than National Median Site EUI" and "Percent Better than National Median Source EUI" are redundant
- We should think about reconstructing the "Total GHG Emissions Intensity" column --> there are missing data points, and others seem to be miscalculated
- **What's the difference between Site and Source EUI?? Here's a helpful link: https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/understand-metrics/difference**. 
- Main takeaway from the above: we should use "source EUI". But we can try generating different models with either one, or both

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('energy_building_data_retabulated.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Create a separate dataframe that only contains values where ENERGY STAR score is not null
data_not_null = data[~data["ENERGY STAR Score"].isnull()]

#dimension of the dataframe
data_not_null.shape

#count the number of records per year
data_not_null[["year", "ENERGY STAR Score"]].groupby(['year']).count()

# Count the number of null records per column
data_not_null.isna().sum()

#Feature Engineering
data_not_null["Previous Year ENERGY STAR Score"] = data_not_null.groupby(["Parcel(s)"])["ENERGY STAR Score"].shift()
data_not_null["Two Years Ago ENERGY STAR Score"] = data_not_null.groupby(["Parcel(s)"])["ENERGY STAR Score"].shift(2)
data_not_null["Current Year YoY Change"] = data_not_null["ENERGY STAR Score"] - data_not_null["Previous Year ENERGY STAR Score"]
data_not_null["Previous Year YoY Change"] = data_not_null["Previous Year ENERGY STAR Score"] - data_not_null["Two Years Ago ENERGY STAR Score"]
data_not_null["Age In Years"] = data_not_null.apply(lambda row: row["year"] - row["Year Built"], axis=1)
data_not_null["Recalculated GHG Emissions Intensity (kgCO2e/ft2)"] = data_not_null.apply(lambda row: row["Total GHG Emissions (Metric Tons CO2e)"] * 1000 / row["Floor Area"], axis=1)
data_not_null["Is Downtown"] = data_not_null.apply(lambda row: True if row["Postal Code"] in [94103, 94104, 94105, 94107, 94108] else False, axis=1)

data_not_null_dropped = data_not_null.drop(columns=["Total GHG Emissions Intensity (kgCO2e/ft2)", "Benchmark Status", 
                            "Reason for Exemption", "PIM Link"]);
data_not_null.to_csv("building_energy_data_final.csv")